In [41]:
# Enables autoreload extension
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
!pip install tabulate

In [45]:
# Imports and Initial Setup


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import data_utils  # custom module in src folder
import common_imports as ci
from tabulate import tabulate

In [3]:
# Loading Data

file_path = "/home/Isha/USD/usd_project_aai_500/AAI-500-IN1_PROJECT/data/Raw/Sales Transaction v.4a.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')
print(df.shape)
df.head()
log_data_cleaning = data_utils.track_observations("df.shape >> There are total 536350 Rows and 8 Columns in dataset", obs_type='finding')
log_data_cleaning

(536350, 8)


[{'observation': 'df.shape >> There are total 536350 Rows and 8 Columns in dataset',
  'type': 'finding'}]

In [4]:
# Initial Inspection

print(df.info())
log_data_cleaning = data_utils.track_observations("df.info() >> Date Datatype is Object", obs_type='finding')

print(df.describe(include='all'))
log_data_cleaning = data_utils.track_observations("df.describe(include='all') >> (NUMERIC COLUMN)Price ->	Shows mean = 12.66, std, min, max → float", obs_type='finding')
log_data_cleaning = data_utils.track_observations("df.describe(include='all') >> (NUMERIC COLUMN)Quantity	-> Shows wide range and stats → integer", obs_type='finding')
log_data_cleaning = data_utils.track_observations("df.describe(include='all') >> (NUMERIC COLUMN)CustomerNo -> Shows mean = 15227, min = 12004 → integer", obs_type='finding')
log_data_cleaning = data_utils.track_observations("df.describe(include='all') >> (STRING COLUMN)CustomerNo -> TransactionNo, Date, ProductNo, ProductName, Country are string ", obs_type='finding')
log_data_cleaning = data_utils.track_observations("df.describe(include='all') >> (STRING COLUMN)CustomerNo -> Date Column is string -> Should be datetime ", obs_type='to_change')

print(df.isnull().sum())
log_data_cleaning = data_utils.track_observations("df.isnull().sum() >> There are 55 null values in CustomerNo column", obs_type='finding')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  object 
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB
None
       TransactionNo       Date ProductNo                         ProductName  \
count         536350     536350    536350                              536350   
unique         23204        305      3768                                3768   
top           573585  12/5/2019    85123A  Cream Hanging Heart T-Light Holder   
freq            1111       5299      2378       

In [5]:
# Unique Values per Column
print(df.nunique())

TransactionNo    23204
Date               305
ProductNo         3768
ProductName       3768
Price              530
Quantity           510
CustomerNo        4738
Country             38
dtype: int64


In [6]:
# Remove Duplicates
df = df.drop_duplicates()
print("After removing duplicates:", df.shape)
log_data_cleaning = data_utils.track_observations("df.drop_duplicates() >> After removing duplicates shape: (531150, 8)", obs_type='finding')


After removing duplicates: (531150, 8)


In [7]:
# Clean Numeric Columns (Price, Quantity)
df_cleaned, df_neg, df_zero = data_utils.clean_numeric_columns(
    df,
    cols=['Price', 'Quantity'],
    remove_zeros=True,
    remove_negatives=True,
    outlier_method='zscore',
    drop_outliers=True
)
print("After cleaning numerics:", df_cleaned.shape)
log_data_cleaning = data_utils.track_observations("data_utils.clean_numeric_columns(df,-,-,,-) >> After cleaning numerics: (521364, 8)", obs_type='finding')
print("Cleaned df:",df_cleaned.shape)
log_data_cleaning = data_utils.track_observations("data_utils.clean_numeric_columns(df,-,-,,-) >> There are 521364 valid positive values for price and quantity", obs_type='finding')
print("Negative values df",df_neg.shape)
log_data_cleaning = data_utils.track_observations("data_utils.clean_numeric_columns(df,-,-,,-) >> There are 8493 negative values for price or quantity", obs_type='finding')
print("Zero values df",df_zero.shape)
log_data_cleaning = data_utils.track_observations("data_utils.clean_numeric_columns(df,-,-,,-) >> There are no 0 value for price and quantity", obs_type='finding')


After cleaning numerics: (521364, 8)
Cleaned df: (521364, 8)
Negative values df (8493, 8)
Zero values df (0, 8)


In [8]:
# Convert and Extract Date Features
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'], errors='coerce')
df_cleaned['Month'] = df_cleaned['Date'].dt.month
df_cleaned['Weekday'] = df_cleaned['Date'].dt.day_name()

df_neg['Date'] = pd.to_datetime(df_neg['Date'], errors='coerce')
df_neg['Month'] = df_neg['Date'].dt.month
df_neg['Weekday'] = df_neg['Date'].dt.day_name()

In [9]:
df_cleaned


TransactionNo       Date ProductNo  \
0             581482 2019-12-09     22485   
1             581475 2019-12-09     22596   
2             581475 2019-12-09     23235   
3             581475 2019-12-09     23272   
4             581475 2019-12-09     23239   
...              ...        ...       ...   
536320        536585 2018-12-01     37449   
536321        536590 2018-12-01     22776   
536322        536590 2018-12-01     22622   
536323        536591 2018-12-01     37449   
536324        536597 2018-12-01     22220   

                                ProductName  Price  Quantity  CustomerNo  \
0             Set Of 2 Wooden Market Crates  21.47        12     17490.0   
1       Christmas Star Wish List Chalkboard  10.65        36     13069.0   
2                  Storage Tin Vintage Leaf  11.53        12     13069.0   
3         Tree T-Light Holder Willie Winkie  10.65        12     13069.0   
4         Set Of 4 Knick Knack Tins Poppies  11.94         6     13069.0   
...                                     ...    ...       ...         ...   
536320   Ceramic Cake Stand + Hanging Cakes  20.45         2     17460.0   
536321         Sweetheart 3 Tier Cake Stand  20.45         1     13065.0   
536322       Box Of Vintage Alphabet Blocks  20.45         2     13065.0   
536323   Ceramic Cake Stand + Hanging Cakes  20.45         1     14606.0   
536324     Cake Stand Lovebird 2 Tier White  20.45         1     18011.0   

               Country  Month   Weekday  
0       United Kingdom     12    Monday  
1       United Kingdom     12    Monday  
2       United Kingdom     12    Monday  
3       United Kingdom     12    Monday  
4       United Kingdom     12    Monday  
...                ...    ...       ...  
536320  United Kingdom     12  Saturday  
536321  United Kingdom     12  Saturday  
536322  United Kingdom     12  Saturday  
536323  United Kingdom     12  Saturday  
536324  United Kingdom     12  Saturday  

[521364 rows x 10 columns]

In [10]:
df_neg

TransactionNo       Date ProductNo                       ProductName  \
1616         C581484 2019-12-09     23843         Paper Craft Little Birdie   
1617         C581490 2019-12-09     22178   Victorian Glass Hanging T-Light   
1618         C581490 2019-12-09     23144   Zinc T-Light Holder Stars Small   
1619         C581568 2019-12-09     21258        Victorian Sewing Box Large   
1620         C581569 2019-12-09     84978  Hanging Heart Jar T-Light Holder   
...              ...        ...       ...                               ...   
536345       C536548 2018-12-01     22168      Organiser Wood Antique White   
536346       C536548 2018-12-01     21218            Red Spotty Biscuit Tin   
536347       C536548 2018-12-01     20957      Porcelain Hanging Bell Small   
536348       C536548 2018-12-01     22580      Advent Calendar Gingham Sack   
536349       C536548 2018-12-01     22767        Triple Photo Frame Cornice   

        Price  Quantity  CustomerNo         Country  Month   Weekday  
1616     6.19    -80995     16446.0  United Kingdom     12    Monday  
1617     6.19       -12     14397.0  United Kingdom     12    Monday  
1618     6.04       -11     14397.0  United Kingdom     12    Monday  
1619     6.19        -5     15311.0  United Kingdom     12    Monday  
1620     6.19        -1     17315.0  United Kingdom     12    Monday  
...       ...       ...         ...             ...    ...       ...  
536345  18.96        -2     12472.0         Germany     12  Saturday  
536346  14.09        -3     12472.0         Germany     12  Saturday  
536347  11.74        -1     12472.0         Germany     12  Saturday  
536348  16.35        -4     12472.0         Germany     12  Saturday  
536349  20.45        -2     12472.0         Germany     12  Saturday  

[8493 rows x 10 columns]

In [14]:
# Check Categorical Variables

## Country
print(df['Country'].value_counts())
log_data_cleaning = data_utils.track_observations("df['Country'].value_counts() >> There are 443 Unspecified values in country column", obs_type='to_change')



Country
United Kingdom          480005
Germany                  10656
France                   10509
EIRE                      8024
Belgium                   2539
Spain                     2426
Netherlands               2330
Switzerland               2330
Portugal                  1840
Australia                 1702
Norway                     938
Austria                    887
Iceland                    785
Finland                    692
Italy                      661
Channel Islands            629
Cyprus                     569
Unspecified                443
Sweden                     417
Denmark                    416
USA                        378
Japan                      371
Israel                     353
Singapore                  215
Poland                     174
Canada                     149
Malta                      149
Hong Kong                  149
Greece                      67
United Arab Emirates        67
European Community          58
RSA                         57


In [15]:
#df_cleaned = data_utils.clean_object_columns(df_cleaned)
# Check if 'Undefined' exists in Country column
df_cleaned[df_cleaned['Country'] == 'Undefined']
log_data_cleaning = data_utils.track_observations("df_cleaned[df_cleaned['Country'] == 'Undefined'] >> There are 0 Unspecified values in country column in df_cleaned", obs_type='to_change')

#df_cleaned[df_cleaned['Country'] == 'Nan']


In [16]:
df_cleaned

TransactionNo       Date ProductNo  \
0             581482 2019-12-09     22485   
1             581475 2019-12-09     22596   
2             581475 2019-12-09     23235   
3             581475 2019-12-09     23272   
4             581475 2019-12-09     23239   
...              ...        ...       ...   
536320        536585 2018-12-01     37449   
536321        536590 2018-12-01     22776   
536322        536590 2018-12-01     22622   
536323        536591 2018-12-01     37449   
536324        536597 2018-12-01     22220   

                                ProductName  Price  Quantity  CustomerNo  \
0             Set Of 2 Wooden Market Crates  21.47        12     17490.0   
1       Christmas Star Wish List Chalkboard  10.65        36     13069.0   
2                  Storage Tin Vintage Leaf  11.53        12     13069.0   
3         Tree T-Light Holder Willie Winkie  10.65        12     13069.0   
4         Set Of 4 Knick Knack Tins Poppies  11.94         6     13069.0   
...                                     ...    ...       ...         ...   
536320   Ceramic Cake Stand + Hanging Cakes  20.45         2     17460.0   
536321         Sweetheart 3 Tier Cake Stand  20.45         1     13065.0   
536322       Box Of Vintage Alphabet Blocks  20.45         2     13065.0   
536323   Ceramic Cake Stand + Hanging Cakes  20.45         1     14606.0   
536324     Cake Stand Lovebird 2 Tier White  20.45         1     18011.0   

               Country  Month   Weekday  
0       United Kingdom     12    Monday  
1       United Kingdom     12    Monday  
2       United Kingdom     12    Monday  
3       United Kingdom     12    Monday  
4       United Kingdom     12    Monday  
...                ...    ...       ...  
536320  United Kingdom     12  Saturday  
536321  United Kingdom     12  Saturday  
536322  United Kingdom     12  Saturday  
536323  United Kingdom     12  Saturday  
536324  United Kingdom     12  Saturday  

[521364 rows x 10 columns]

In [17]:
## Product Columns
# Top 10 Products - count
print("TOP 10 PRODUCTS: \n",df_cleaned['ProductName'].value_counts().head(10))

# Get uniqe products with less than 5 count with count
## counting frequency of each product
product_counts = df_cleaned['ProductName'].value_counts()
## Filter products that appear less than 5 times
rare_products = product_counts[product_counts < 5]
## Convert to DataFrame for better readability
rare_products_df = rare_products.reset_index()
rare_products_df.columns = ['ProductName', 'Count']
print("\nRARE PRODUCTS COUNT: \n",rare_products_df["ProductName"].count())
print("\nRARE PRODUCTS: \n",rare_products_df.head())
print("\nRARE PRODUCTS: \n",rare_products_df.tail())

TOP 10 PRODUCTS: 
 ProductName
Cream Hanging Heart T-Light Holder    2317
Jumbo Bag Red Retrospot               2109
Regency Cakestand 3 Tier              2010
Party Bunting                         1701
Lunch Bag Red Retrospot               1584
Assorted Colour Bird Ornament         1475
Popcorn Holder                        1400
Set Of 3 Cake Tins Pantry Design      1391
Pack Of 72 Retrospot Cake Cases       1349
Lunch Bag Suki Design                 1308
Name: count, dtype: int64

RARE PRODUCTS COUNT: 
 375

RARE PRODUCTS: 
                           ProductName  Count
0  Set 10 Cards David's Madonna 17074      4
1               Hanging Butterfly Egg      4
2                Robin Christmas Card      4
3            Diamante Necklace Purple      4
4  Set 10 Card Perfect Nativity 17089      4

RARE PRODUCTS: 
                              ProductName  Count
370              Ruby Glass Necklace 42"      1
371            Hen House W Chick In Nest      1
372          Purple/Turq Flowers Ha

In [19]:
# Feature Engineering
df_cleaned['TotalPrice'] = df_cleaned['Quantity'] * df_cleaned['Price']
df_cleaned

TransactionNo       Date ProductNo  \
0             581482 2019-12-09     22485   
1             581475 2019-12-09     22596   
2             581475 2019-12-09     23235   
3             581475 2019-12-09     23272   
4             581475 2019-12-09     23239   
...              ...        ...       ...   
536320        536585 2018-12-01     37449   
536321        536590 2018-12-01     22776   
536322        536590 2018-12-01     22622   
536323        536591 2018-12-01     37449   
536324        536597 2018-12-01     22220   

                                ProductName  Price  Quantity  CustomerNo  \
0             Set Of 2 Wooden Market Crates  21.47        12     17490.0   
1       Christmas Star Wish List Chalkboard  10.65        36     13069.0   
2                  Storage Tin Vintage Leaf  11.53        12     13069.0   
3         Tree T-Light Holder Willie Winkie  10.65        12     13069.0   
4         Set Of 4 Knick Knack Tins Poppies  11.94         6     13069.0   
...                                     ...    ...       ...         ...   
536320   Ceramic Cake Stand + Hanging Cakes  20.45         2     17460.0   
536321         Sweetheart 3 Tier Cake Stand  20.45         1     13065.0   
536322       Box Of Vintage Alphabet Blocks  20.45         2     13065.0   
536323   Ceramic Cake Stand + Hanging Cakes  20.45         1     14606.0   
536324     Cake Stand Lovebird 2 Tier White  20.45         1     18011.0   

               Country  Month   Weekday  TotalPrice  
0       United Kingdom     12    Monday      257.64  
1       United Kingdom     12    Monday      383.40  
2       United Kingdom     12    Monday      138.36  
3       United Kingdom     12    Monday      127.80  
4       United Kingdom     12    Monday       71.64  
...                ...    ...       ...         ...  
536320  United Kingdom     12  Saturday       40.90  
536321  United Kingdom     12  Saturday       20.45  
536322  United Kingdom     12  Saturday       40.90  
536323  United Kingdom     12  Saturday       20.45  
536324  United Kingdom     12  Saturday       20.45  

[521364 rows x 11 columns]

In [20]:
# Handle Missing Values
# Replace missing values with 'NA'
df_cleaned['CustomerNo'] = df_cleaned['CustomerNo'].fillna('NA')
log_data_cleaning = data_utils.track_observations("Handling Missing Values >> Convert all missing values to NA", obs_type='to_change')

# Convert all to string first (safe for mixed types)
df_cleaned['CustomerNo'] = df_cleaned['CustomerNo'].astype(str)
log_data_cleaning = data_utils.track_observations("Change customerno to int type >> Convert custumerno to string type", obs_type='to_change')

# Convert numeric entries to float, leave 'NA' as-is
df_cleaned['CustomerNo'] = df_cleaned['CustomerNo'].apply(lambda x: float(x) if x != 'NA' else 'NA')
log_data_cleaning = data_utils.track_observations("Change customerno to int type >> Convert string to int type conditionally", obs_type='to_change')


In [28]:
df_cleaned
df_cleaned[df_cleaned['CustomerNo'].isna()]
log_data_cleaning = data_utils.track_observations("df_cleaned[df_cleaned['CustomerNo'].isna()] >> there is 0 missing record", obs_type='finding')
df_cleaned[df_cleaned['CustomerNo'] == 'NA']
print((df_cleaned['CustomerNo'] == 'NA').sum())
log_data_cleaning = data_utils.track_observations("df_cleaned[df_cleaned['CustomerNo'] == 'NA'] >> there is 1 record change to NA", obs_type='finding')


1


In [31]:
# Final Output
print(df_cleaned.head())
print(df_cleaned.shape)

log_data_cleaning = data_utils.track_observations("df_cleaned shape = (521364, 11)", obs_type='finding')


  TransactionNo       Date ProductNo                          ProductName  \
0        581482 2019-12-09     22485        Set Of 2 Wooden Market Crates   
1        581475 2019-12-09     22596  Christmas Star Wish List Chalkboard   
2        581475 2019-12-09     23235             Storage Tin Vintage Leaf   
3        581475 2019-12-09     23272    Tree T-Light Holder Willie Winkie   
4        581475 2019-12-09     23239    Set Of 4 Knick Knack Tins Poppies   

   Price  Quantity CustomerNo         Country  Month Weekday  TotalPrice  
0  21.47        12    17490.0  United Kingdom     12  Monday      257.64  
1  10.65        36    13069.0  United Kingdom     12  Monday      383.40  
2  11.53        12    13069.0  United Kingdom     12  Monday      138.36  
3  10.65        12    13069.0  United Kingdom     12  Monday      127.80  
4  11.94         6    13069.0  United Kingdom     12  Monday       71.64  
(521364, 11)


In [30]:
df_cleaned.head()

TransactionNo       Date ProductNo                          ProductName  \
0        581482 2019-12-09     22485        Set Of 2 Wooden Market Crates   
1        581475 2019-12-09     22596  Christmas Star Wish List Chalkboard   
2        581475 2019-12-09     23235             Storage Tin Vintage Leaf   
3        581475 2019-12-09     23272    Tree T-Light Holder Willie Winkie   
4        581475 2019-12-09     23239    Set Of 4 Knick Knack Tins Poppies   

   Price  Quantity CustomerNo         Country  Month Weekday  TotalPrice  
0  21.47        12    17490.0  United Kingdom     12  Monday      257.64  
1  10.65        36    13069.0  United Kingdom     12  Monday      383.40  
2  11.53        12    13069.0  United Kingdom     12  Monday      138.36  
3  10.65        12    13069.0  United Kingdom     12  Monday      127.80  
4  11.94         6    13069.0  United Kingdom     12  Monday       71.64

In [50]:
log_data_cleaning
# Convert to DataFrame
df_log = pd.DataFrame(log_data_cleaning)

# Print as a table using tabulate
print("\n📋 Data Cleaning Log Table:\n")

pd.set_option('display.max_colwidth', None)
#display(df_log)
df_log.style.set_properties(**{'text-align': 'left'})


📋 Data Cleaning Log Table:

